In [2]:
!pip install dash
!pip install dash-core-components
!pip install dash-html-components
!pip install dash-renderer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-py3-none-any.whl size=1014851 sha256=2a18c5fc7dc6f609e286ae37edf8933f153ab0c4e1c876ec9975b24976a46

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-3-89e3ae8eb04c>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-3-89e3ae8eb04c>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [4]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [5]:
df['Launch Site'].value_counts()

CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: Launch Site, dtype: int64

In [6]:
import plotly.graph_objects as go

In [7]:
import requests

url1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
response = requests.get(url1)

with open("spacex_launch_dash.csv", "w") as file:
    file.write(response.text)

print("Sapce X launch dash downloaded successfully.")

Sapce X launch dash downloaded successfully.


In [20]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Convert min_payload and max_payload to integers
min_payload = int(min_payload)
max_payload = int(max_payload)

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': 'white',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(
                                id='site-dropdown',
                                options=[
                                  {'label': 'All Sites', 'value': 'ALL'},
                                  {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                  {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                  {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                  {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                ],
                                value='ALL',
                                placeholder='Select a Launch Site here',
                                searchable=True
                                ),
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                 # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=min_payload,
                                    max=max_payload,
                                    step=1000,
                                    marks={i: str(i) for i in range(min_payload, max_payload+1, 1000)},
                                    value=[min_payload, max_payload]
                                ),
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2: Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
    # Logic to create pie chart for all sites
        success_percentage_df = spacex_df.groupby('Launch Site')['class'].mean() * 100
        fig = go.Figure(data=[go.Pie(labels=success_percentage_df.index, values=success_percentage_df.values)])
        fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=10)
        fig.update_layout(title_text='Success Percentage Distribution for All Sites')
        return fig


    else:
        # Logic to create pie chart for a specific site
        specific_site_count = spacex_df[spacex_df['Launch Site'] == selected_site]['class'].value_counts()
        fig = go.Figure(data=[go.Pie(labels=['Successful', 'Failed'], values=specific_site_count)])
        fig.update_layout(title_text=f'Success vs Failed Launches for {selected_site}')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(site, payload_range):
    filtered_df = spacex_df
    if site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == site]
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= payload_range[0]) & (filtered_df['Payload Mass (kg)'] <= payload_range[1])]
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    fig.update_layout(
        title='Correlation between Payload and Launch Success',
        xaxis_title='Payload Mass (kg)',
        yaxis_title='Launch Success',
        legend_title='Booster Version Category'
    )
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server()


<IPython.core.display.Javascript object>